<a href="https://colab.research.google.com/github/ssravali/Research-1/blob/master/Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Libraries initiated
import numpy as np
import pickle
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import logging
from time import time
import pandas as pd
from keras.layers import Flatten
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import sequence
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.layers import Conv1D, Dropout
from keras.layers import MaxPooling1D
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
import scipy.io as sio
import keras
import random
print("Imported libraries")

Using TensorFlow backend.


Imported libraries


In [0]:
mat_contents = sio.loadmat('augtestx2.mat')
testx = np.array(mat_contents['test_x'])
test_x = testx[0:120]

mat_contents = sio.loadmat('augtesty2.mat')
testy = np.array(mat_contents['test_y'])
test_y = testy[0:120]

mat_contents = sio.loadmat('augtrainx2.mat')
train_x = np.array(mat_contents['train_x'])

mat_contents = sio.loadmat('augtrainy2.mat')
trainy = np.array(mat_contents['train_y'])

classes = np.unique(trainy)
nClasses = len(classes)

val_x1 = []
val_x2 = []
val_y1 = []
val_y2 = []

for i in range(nClasses):
    #print("Class: ", i)
    j = 2
    r,r1 = np.where(trainy == i)
    while(j <= (r.shape[0]/2)):
        val_x2.append(train_x[r[j]])
        val_y2.append(trainy[r[j]])
        j = j + 3
    r,r1 = np.where(test_y == i)
    j = 2
    k = 5
    R = []
    while(k <= r.shape[0]):
        val_x1.append(test_x[r[j]])
        val_y1.append(test_y[r[j]])
        R.append(r[k])
        j = j + 6
        k = k + 6
    train_x = np.concatenate((train_x, test_x[R]),axis= 0)
    trainy = np.concatenate((trainy,testy[R]), axis = 0)

val_x1 = np.array(val_x1)
val_x2 = np.array(val_x2)
val_y1 = np.array(val_y1, dtype = int)
val_y2 = np.array(val_y2, dtype = int)
val_x = np.concatenate((val_x1,val_x2), axis = 0)
val_y = np.concatenate((val_y1,val_y2), axis = 0)

enc = OneHotEncoder(sparse=False,categories='auto')
train_y = enc.fit_transform(trainy)
test_y = enc.fit_transform(test_y)
val_y = enc.fit_transform(val_y)

print("Train_y one hot:", train_y.shape)
print("Test_y one hot:", test_y.shape)
print("Val_y one hot:", val_y.shape)


print(train_x.shape)
print(test_x.shape)
print(val_x.shape)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
print("loaded data")


Train_y one hot: (980, 10)
Test_y one hot: (120, 10)
Val_y one hot: (180, 10)
(980, 800, 10)
(120, 800, 10)
(180, 800, 10)
Total number of outputs :  10
Output classes :  [0 1 2 3 4 5 6 7 8 9]
loaded data


In [0]:
mat_contents = sio.loadmat('augtestx2.mat')
test_x = np.array(mat_contents['test_x'])

mat_contents = sio.loadmat('augtesty2.mat')
test_y = np.array(mat_contents['test_y'])

mat_contents = sio.loadmat('augtrainx2.mat')
train_x = np.array(mat_contents['train_x'])

mat_contents = sio.loadmat('augtrainy2.mat')
train_y = np.array(mat_contents['train_y'])

classes = np.unique(train_y)
nClasses = len(classes)

x_train = []
y_train = []
x_test = []
y_test = []
x_val = []
y_val = []
for i in classes:
    #print("i is: ",i)
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    r, r1 = np.where(train_y == i)
    while(o < r.shape[0]):
        x_train.append(train_x[r[j]])
        x_train.append(train_x[r[k]])
        x_train.append(train_x[r[l]])
        x_train.append(train_x[r[m]])
        x_test.append(train_x[r[n]])
        x_val.append(train_x[r[o]])
        #x_test.append(train_x[r[o]])
        y_train.append(train_y[r[j]])
        y_train.append(train_y[r[k]])
        y_train.append(train_y[r[l]])
        y_train.append(train_y[r[m]])
        y_test.append(train_y[r[n]])
        y_val.append(train_y[r[o]])
        #y_test.append(train_y[r[o]])
        j = j + 6
        k = k + 6
        l = l + 6
        m = m + 6
        n = n + 6
        o = o + 6
    #print("len of y_test:",len(y_test))
    #print("len y_train: ",len(y_train))
    #print("len y_val: ",len(y_val))
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    r, r1 = np.where(test_y == i)
    while(o < r.shape[0]):
        x_train.append(test_x[r[j]])
        x_train.append(test_x[r[k]])
        x_train.append(test_x[r[l]])
        x_train.append(test_x[r[m]])
        x_test.append(test_x[r[n]])
        x_val.append(test_x[r[o]])
        #x_test.append(train_x[r[o]])
        y_train.append(test_y[r[j]])
        y_train.append(test_y[r[k]])
        y_train.append(test_y[r[l]])
        y_train.append(test_y[r[m]])
        y_test.append(test_y[r[n]])
        y_val.append(test_y[r[o]])
        #y_test.append(train_y[r[o]])
        j = j + 6
        k = k + 6
        l = l + 6
        m = m + 6
        n = n + 6
        o = o + 6
    #print("len of y_test:",len(y_test))
    #print("len y_train: ",len(y_train))
    #print("len y_val: ",len(y_val))
    
    

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)



(800, 800, 10)
(800, 1)
(200, 800, 10)
(200, 1)
(200, 800, 10)
(200, 1)


Set1 where test dataset also has agumented data

In [0]:
mat_contents = sio.loadmat('augtestx2.mat')
test_x = np.array(mat_contents['test_x'])

mat_contents = sio.loadmat('augtesty2.mat')
test_y = np.array(mat_contents['test_y'])

mat_contents = sio.loadmat('augtrainx2.mat')
train_x = np.array(mat_contents['train_x'])

mat_contents = sio.loadmat('augtrainy2.mat')
train_y = np.array(mat_contents['train_y'])

classes = np.unique(train_y)
nClasses = len(classes)

x_train = []
y_train = []
x_test = []
y_test = []
x_val = []
y_val = []
for i in classes:
    #print("i is: ",i)
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    r, r1 = np.where(train_y == i)
    while(o < r.shape[0]):
        x_train.append(train_x[r[j]])
        x_train.append(train_x[r[k]])
        x_train.append(train_x[r[l]])
        x_train.append(train_x[r[m]])
        x_val.append(train_x[r[o]])
        if(o < r.shape[0]/2):
            x_test.append(train_x[r[n]])
            y_test.append(train_y[r[n]])
        #x_test.append(train_x[r[o]])
        y_train.append(train_y[r[j]])
        y_train.append(train_y[r[k]])
        y_train.append(train_y[r[l]])
        y_train.append(train_y[r[m]])
        y_val.append(train_y[r[o]])
            
        
        #y_test.append(train_y[r[o]])
        j = j + 6
        k = k + 6
        l = l + 6
        m = m + 6
        n = n + 6
        o = o + 6
    #print("len of y_test:",len(y_test))
    #print("len y_train: ",len(y_train))
    #print("len y_val: ",len(y_val))
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    r, r1 = np.where(test_y == i)
    while(o < r.shape[0]):
        x_train.append(test_x[r[j]])
        x_train.append(test_x[r[k]])
        x_train.append(test_x[r[l]])
        x_train.append(test_x[r[m]])
        x_val.append(test_x[r[o]])
        if(o < r.shape[0]/2):
            x_test.append(test_x[r[n]])
            y_test.append(test_y[r[n]])
        #x_test.append(train_x[r[o]])
        y_train.append(test_y[r[j]])
        y_train.append(test_y[r[k]])
        y_train.append(test_y[r[l]])
        y_train.append(test_y[r[m]])
        y_val.append(test_y[r[o]])
        #y_test.append(train_y[r[o]])
        j = j + 6
        k = k + 6
        l = l + 6
        m = m + 6
        n = n + 6
        o = o + 6
    #print("len of y_test:",len(y_test))
    #print("len y_train: ",len(y_train))
    #print("len y_val: ",len(y_val))
    
    

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)



(800, 800, 10)
(800, 1)
(100, 800, 10)
(100, 1)
(200, 800, 10)
(200, 1)


In [0]:
Set1 where test data doesn't have agumented data

In [0]:
mat_contents = sio.loadmat('augtestx2.mat')
test_x = np.array(mat_contents['test_x'])

mat_contents = sio.loadmat('augtesty2.mat')
test_y = np.array(mat_contents['test_y'])

mat_contents = sio.loadmat('augtrainx2.mat')
train_x = np.array(mat_contents['train_x'])

mat_contents = sio.loadmat('augtrainy2.mat')
train_y = np.array(mat_contents['train_y'])

classes = np.unique(train_y)
nClasses = len(classes)

x_val = []
y_val = []

for i in classes:
    #print("i is: ",i)
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    
    r, r1 = np.where(test_y == i)
    while(o < r.shape[0]):
        y_val.append(test_y[r[j]])
        y_val.append(test_y[r[k]])
        y_val.append(test_y[r[l]])
        y_val.append(test_y[r[m]])
        y_val.append(test_y[r[n]])
        y_val.append(test_y[r[o]])
        x_val.append(test_x[r[j]])
        x_val.append(test_x[r[k]])
        x_val.append(test_x[r[l]])
        x_val.append(test_x[r[m]])
        x_val.append(test_x[r[n]])
        x_val.append(test_x[r[o]])
        
        j = j + 12
        k = k + 12
        l = l + 12
        m = m + 12
        n = n + 12
        o = o + 12

x_val = np.array(x_val)
y_val = np.array(y_val)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(x_val.shape)
print(y_val.shape)


(960, 800, 10)
(960, 1)
(240, 800, 10)
(240, 1)
(120, 800, 10)
(120, 1)


Set to aug

In [0]:
mat_contents = sio.loadmat('augtestx2.mat')
test_x = np.array(mat_contents['test_x'])

mat_contents = sio.loadmat('augtesty2.mat')
test_y = np.array(mat_contents['test_y'])

mat_contents = sio.loadmat('augtrainx2.mat')
train_x = np.array(mat_contents['train_x'])

mat_contents = sio.loadmat('augtrainy2.mat')
train_y = np.array(mat_contents['train_y'])

classes = np.unique(train_y)
nClasses = len(classes)

x_val = []
y_val = []
x_test = []
y_test = []

for i in classes:
    #print("i is: ",i)
    j = 0
    k = 1
    l = 2
    m = 3
    n = 4
    o = 5
    
    r, r1 = np.where(test_y == i)
    while(o < r.shape[0]):
        y_test.append(test_y[r[j]])
        y_test.append(test_y[r[k]])
        y_test.append(test_y[r[l]])
        y_val.append(test_y[r[m]])
        y_val.append(test_y[r[n]])
        y_val.append(test_y[r[o]])
        x_test.append(test_x[r[j]])
        x_test.append(test_x[r[k]])
        x_test.append(test_x[r[l]])
        x_val.append(test_x[r[m]])
        x_val.append(test_x[r[n]])
        x_val.append(test_x[r[o]])
        
        j = j + 6
        k = k + 6
        l = l + 6
        m = m + 6
        n = n + 6
        o = o + 6

x_val = np.array(x_val)
y_val = np.array(y_val)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(train_x.shape)
print(train_y.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)


(960, 800, 10)
(960, 1)
(120, 800, 10)
(120, 1)
(120, 800, 10)
(120, 1)
